SLiMe is a package that allows the user to simulate large-scale genomic data under realistically complex demographic histories while retaining information about local ancestry.

This notebook describes how to use SLiMe.

### Basic procedure

1. Recent history since the start of admixture is simulated forward-in-time with `SLiM`.
2. Older history of the ancestral populations is simulated backwards-in-time with `msprime`.

## Necessary inputs

**Recombination map**

A recombination map for the admixed history can be given. 
The file should look like the one below.
Ensure that the final row of the recombination map corresponds to the length of the chromosome that you wish to simulate.

```
Chromosome position COMBINED_rate(cM/Mb) Genetic_Map(cM)
22 16051347 9.6640973708 0
22 16052618 9.7078062447 0.0123386217370137
22 16053624 9.7138922111 0.0221107973013803
22 16053659 9.716343506 0.0224508693240903
22 16053758 9.707808785 0.0234119423938053
22 16054713 9.5801157234 0.0325609529096523
22 16054960 9.5805112114 0.0349273391788681
.
.
.
22 49991555 0.1286194771 73.1460580463483
22 49991660 0.1286937712 73.1460715591943
22 49992695 0.1297177603 73.1462058170762
22 49993745 0.1310658564 73.1463434362254
22 49994152 0.1310658564 73.146396780029
22 49997158 0.1310658564 73.1467907639933
22 49997614 0.1310667948 73.1468505304517
22 49997706 0.1313729993 73.1468626167677
22 49997967 0.1313729993 73.1468969051205
22 50000000 0.1315909877 73.1470166529193
```

### Packages

In [1]:
import slime

## Step 1: Simulate history forward-in-time with SLiM.

See the [SLiM Manual](http://benhaller.com/slim/SLiM_Manual.pdf) for how to set up your simulation.



In [2]:
slim_script = '''
// set up a simple neutral simulation
initialize() {

	setwd("/Users/gtsambos/Projects/AdmixtureSim/5.msprime.and.slim");
	
	// Set constants.
	defineConstant("chromLength", 49999999);
	defineConstant("p0Size", N);
	defineConstant("p1Size", N);
	defineConstant("p2Size", N);
	defineConstant("p0Prop", P/100);
	defineConstant("p1Prop", 1 - p0Prop);

	initializeSLiMModelType("WF");
	initializeSLiMOptions(keepPedigrees = T); // Keep pedigree information
	initializeTreeSeq();
	initializeMutationRate(0);
	
	// m1 mutation type: neutral
	initializeMutationType("m1", 0.5, "f", 0.0);
	
	// g1 genomic element type: uses m1 for all mutations
	initializeGenomicElementType("g1", m1, 1.0);
	
	// uniform chromosome of length 100 kb
	initializeGenomicElement(g1, 0, chromLength);
	
	// read in chr22 recombination map.
	// See example 6.1.2 in the SLiM manual.
	lines = readFile("refs/chr22.recombination_map");
	lines = lines[1:(size(lines)-1)]; // remove header
	rates = NULL;
	ends = NULL;
	
	for (line in lines)
	{
		components = strsplit(line, " ");
		ends = c(ends, asInteger(components[1]));
		rates = c(rates, asFloat(components[2]));
	}
	
	ends = ends - 1; // final coordinate in file is 50 000 000
	rates = rates * 1e-8;
	
	initializeRecombinationRate(rates, ends);
	
}

1 early(){
	sim.addSubpop("p0", p0Size);
	sim.addSubpop("p1", p1Size);

//	sim.treeSeqRememberIndividuals(sim.subpopulations.individuals);
}

1 late() { 
	
	// Add admixing population.
	sim.addSubpop("p2", p2Size);
	p2.setMigrationRates(c(p0,p1),c(p0Prop, p1Prop));
	
}


2 late() {

	p2.setMigrationRates(c(p0,p1),c(0.0,0.0));

}


// save the tree output after 20 generations.
20 late() {
 	sim.treeSeqOutput("script-1.trees");
	sim.simulationFinished();
}
'''

slim_file = open("examples/slim_script.slim", "w")
slim_file.writelines(slim_script)
slim_file.close()

In [3]:
slime.simulate_recent_history("examples/slim_script.slim")

// Initial random seed:

1570625682360



// RunInitializeCallbacks():

ERROR (EidosSymbolTable::_GetValue): undefined identifier N.



Error on script line 9, character 26:



   defineConstant("p0Size", N);

                            ^



Oops, a mistake! Let's fix up our script and try again.

In [4]:
slim_script = '''
// set up a simple neutral simulation
initialize() {

//	setwd("/Users/gtsambos/Projects/AdmixtureSim/5.msprime.and.slim");
	
	// Set constants.
	defineConstant("chromLength", 49999999);
	defineConstant("p0Size", 10000);
	defineConstant("p1Size", 10000);
	defineConstant("p2Size", 10000);
	defineConstant("p0Prop", 10/100);
	defineConstant("p1Prop", 1 - p0Prop);

	initializeSLiMModelType("WF");
	initializeSLiMOptions(keepPedigrees = T); // Keep pedigree information
	initializeTreeSeq();
	initializeMutationRate(0);
	
	// m1 mutation type: neutral
	initializeMutationType("m1", 0.5, "f", 0.0);
	
	// g1 genomic element type: uses m1 for all mutations
	initializeGenomicElementType("g1", m1, 1.0);
	
	// uniform chromosome of length 100 kb
	initializeGenomicElement(g1, 0, chromLength);
	
	// read in chr22 recombination map.
	// See example 6.1.2 in the SLiM manual.
	lines = readFile("examples/chr22.recombination_map");
	lines = lines[1:(size(lines)-1)]; // remove header
	rates = NULL;
	ends = NULL;
	
	for (line in lines)
	{
		components = strsplit(line, " ");
		ends = c(ends, asInteger(components[1]));
		rates = c(rates, asFloat(components[2]));
	}
	
	ends = ends - 1; // final coordinate in file is 50 000 000
	rates = rates * 1e-8;
	
	initializeRecombinationRate(rates, ends);
	
}

1 early(){
	sim.addSubpop("p0", p0Size);
	sim.addSubpop("p1", p1Size);

//	sim.treeSeqRememberIndividuals(sim.subpopulations.individuals);
}

1 late() { 
	
	// Add admixing population.
	sim.addSubpop("p2", p2Size);
	p2.setMigrationRates(c(p0,p1),c(p0Prop, p1Prop));
	
}


2 late() {

	p2.setMigrationRates(c(p0,p1),c(0.0,0.0));

}


// save the tree output after 20 generations.
200 late() {
 	sim.treeSeqOutput("examples/script-1.trees");
	sim.simulationFinished();
}
'''

slim_file = open("examples/slim_script.slim", "w")
slim_file.writelines(slim_script)
slim_file.close()

In [5]:
slime.simulate_recent_history("examples/slim_script.slim")

Cool. Time to move on.